### 프로젝트 종합실습
##### daum.net 금융 페이지에 접속하여 지난 10년 간의 코스피/환율 데이터를 대량으로 수집하고, 이를 바탕으로 간단한 시각화를 해 보세요. 코스피와 환율 데이터를 선그래프로 표현하여 서로 어떤 상관관계를 보이는지 추세를 보여주면 됩니다.
##### 코스피 데이터 수집 링크 : https://finance.daum.net/api/market_index/days
##### 환율 데이터 수집 링크 : https://finance.daum.net/api/exchanges/FRX.KRWUSD/days

1. 코스피 데이터 수집

In [ ]:
import requests
import pandas as pd

header = {
    "referer": "https://finance.daum.net/domestic/kospi",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"
}

result_kospi = []
url_kospi = "https://finance.daum.net/api/market_index/days"
i = 0
for page_no in range(1, 51):
    i += 1
    print("코스피 {}번째 페이지를 크롤링중입니다...".format(i), end="\r")
    param_kospi = {
        "page": page_no,
        "perPage": "100",
        "market": "KOSPI",
        "pagination": "true"
    }
    res_kospi = requests.get(url_kospi, headers=header, params=param_kospi, verify=False)
    res_kospi_parsed = res_kospi.json()
    for item in res_kospi_parsed["data"]:
        this_dic = {
            "date" : item["date"][:10],
            "kospi" : item["tradePrice"]
        }
        result_kospi.append(this_dic)
df_kospi = pd.DataFrame(result_kospi)
df_kospi.sort_values(by="date", inplace=True)
df_kospi.reset_index(inplace=True, drop=True)
df_kospi.plot()
df_kospi

2. 환율 데이터 수집

In [ ]:
result_cr = []
url_cr = "https://finance.daum.net/api/exchanges/FRX.KRWUSD/days"
i = 0
for page_no in range(1, 51):
    i += 1
    print("환율 {}번째 페이지를 크롤링중입니다...".format(i), end="\r")
    param_cr = {
        "symbolCode": "FRX.KRWUSD",
        "terms": "days",
        "page": page_no,
        "perPage": "100"
    }
    res_cr = requests.get(url_cr, headers=header, params=param_cr, verify=False)
    res_cr_parsed = res_cr.json()
    for item in res_cr_parsed["data"]:
        this_dic = {
            "date" : item["date"][:10],
            "cr" : item["basePrice"]
        }
        result_cr.append(this_dic)
df_cr = pd.DataFrame(result_cr)
df_cr.sort_values(by="date", inplace=True)
df_cr.reset_index(inplace=True, drop=True)
df_cr.plot()
df_cr

3. 수집한 코스피/환율 데이터를 하나의 df로 병합(merging)

In [ ]:
df_merged = df_kospi.merge(df_cr, on="date")
df_merged.plot()
df_merged

4. 코스피/환율 컬럼을 표준화하여 그래프로 표현

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
np_data = np.array(df_merged.loc[:, ["kospi", "cr"]])
scaled = scaler.fit(np_data).transform(np_data)

df_merged["kospi_scaled"] = scaled[:, 0]
df_merged["cr_scaled"] = scaled[:, 1]

df_merged.loc[:, ["kospi_scaled", "cr_scaled"]].plot()
df_merged